In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, month

In [2]:
import os

spark = SparkSession.builder.appName('Optimize I').getOrCreate()

base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3])

answers_input_path = os.path.join(project_path, 'data/answers')

questions_input_path = os.path.join(project_path, 'data/questions')

22/06/11 09:43:09 WARN Utils: Your hostname, Linuxclevo resolves to a loopback address: 127.0.1.1; using 10.0.1.137 instead (on interface wlp2s0)
22/06/11 09:43:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/11 09:43:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#spark.conf.set("spark.sql.adaptive.enabled",False)
#spark.conf.get("spark.sql.adaptive.enabled")

In [4]:
#job 0 init
answersDF = spark.read.option('path', answers_input_path).load()

In [5]:
#job 1 init
questionsDF = spark.read.option('path', questions_input_path).load()

In [6]:
#job 1 6
#questionsDF = spark.read.option('path', questions_input_path).load().coalesce(3)

In [7]:
#job 2 init
answers_month = answersDF.withColumn('month', month('creation_date')) \
    .groupBy('question_id', 'month').agg(count('*').alias('cnt'))

In [8]:
#job 2 1
#answers_month = answersDF.select('question_id','answer_id','creation_date','comments','user_id','score', \
#                                 month(col('creation_date')).alias('month')). \
#                                groupBy('question_id', 'month').agg(count('question_id').alias('cnt'))

In [9]:
#job 2 2
#answers_month = answersDF.repartition('question_id').withColumn('month', month('creation_date')). \
#    groupBy('question_id', 'month').agg(count('*').alias('cnt'))

In [10]:
#job 2 3
#answers_month = answersDF.withColumn('month', month('creation_date')).groupBy('question_id', 'month'). \
#    agg(count('*').alias('cnt')).coalesce(4)

In [11]:
#job 2 me
#answers_month = answersDF.withColumn('month', month('creation_date')).groupBy('question_id', 'month'). \
#    agg(count('*').alias('cnt')).coalesce(3)
#answers_month = answersDF.withColumn('month', month('creation_date')).groupBy('question_id', 'month'). \
#    agg(count('*').alias('cnt'))

In [12]:
#job 3 init
resultDF = questionsDF.join(answers_month, 'question_id') \
    .select('question_id', 'creation_date', 'title', 'month', 'cnt')

In [13]:
#job 3 
#answers_month.cache()
#resultDF = questionsDF.join(answers_month, 'question_id') \
#    .select('question_id', 'creation_date', 'title', 'month', 'cnt')

In [ ]:
#job 4
#questionsDF.cache()
#resultDF = questionsDF.join(answers_month, 'question_id') \
#    .select('question_id', 'creation_date', 'title', 'month', 'cnt')

In [ ]:
#job 5
#answers_month.cache()
#questionsDF.cache()
#resultDF = questionsDF.join(answers_month, 'question_id') \
#    .select('question_id', 'creation_date', 'title', 'month', 'cnt')

In [14]:
#job 3 1
#resultDF = answers_month.join(questionsDF, 'question_id', how='left'). \
#    select('question_id', 'creation_date', 'title', 'month', 'cnt')

In [15]:
#job 4 init
resultDF.orderBy('question_id', 'month').show()

+-----------+--------------------+--------------------+-----+---+
|question_id|       creation_date|               title|month|cnt|
+-----------+--------------------+--------------------+-----+---+
|     155989|2014-12-31 17:59:...|Frost bubble form...|    2|  1|
|     155989|2014-12-31 17:59:...|Frost bubble form...|   12|  1|
|     155990|2014-12-31 18:51:...|The abstract spac...|    1|  1|
|     155990|2014-12-31 18:51:...|The abstract spac...|   12|  1|
|     155992|2014-12-31 19:44:...|centrifugal force...|   12|  1|
|     155993|2014-12-31 19:56:...|How can I estimat...|    1|  1|
|     155995|2014-12-31 21:16:...|Why should a solu...|    1|  3|
|     155996|2014-12-31 22:06:...|Why do we assume ...|    1|  2|
|     155996|2014-12-31 22:06:...|Why do we assume ...|    2|  1|
|     155996|2014-12-31 22:06:...|Why do we assume ...|   11|  1|
|     155997|2014-12-31 22:26:...|Why do square sha...|    1|  3|
|     155999|2014-12-31 23:01:...|Diagonalizability...|    1|  1|
|     1560

In [16]:
#answers_month = answers_month.repartition(4)
#answers_month.rdd.getNumPartitions()
#questionsDF.rdd.getNumPartitions()